Model Serialization And Deserialization

Instance Serialization To Dict And JSON

In [2]:
from datetime import date
from pydantic import BaseModel, model_validator

class ScheduledCourse(BaseModel):
    department: str
    course_number: int
    start_date: date
    end_date: date
    
    @model_validator(mode="after")
    def validate_dates(self):
        if self.start_date > self.end_date:
            raise ValueError("Start date must be before end date.")
        return self

        
course = ScheduledCourse(
    department="CS",
    course_number=101,
    start_date="2025-01-23",
    end_date="2025-09-23"
)

dictCourse = course.model_dump()
print(dictCourse)
jsonCourse = course.model_dump_json(indent=2)
print(jsonCourse)

{'department': 'CS', 'course_number': 101, 'start_date': datetime.date(2025, 1, 23), 'end_date': datetime.date(2025, 9, 23)}
{
  "department": "CS",
  "course_number": 101,
  "start_date": "2025-01-23",
  "end_date": "2025-09-23"
}


Field Exclusions

In [4]:
from datetime import date
from pydantic import BaseModel, model_validator

class ScheduledCourse(BaseModel):
    department: str
    course_number: int
    start_date: date
    end_date: date
    
    @model_validator(mode="after")
    def validate_dates(self):
        if self.start_date > self.end_date:
            raise ValueError("Start date must be before end date.")
        return self

        
course = ScheduledCourse(
    department="CS",
    course_number=101,
    start_date="2025-01-23",
    end_date="2025-09-23"
)

dictCourse = course.model_dump()
print(dictCourse)
jsonCourse = course.model_dump_json(indent=2, exclude={"department", "course_number"})
print(jsonCourse)
jsonCourse = course.model_dump_json(indent=2, include={"department", "course_number"})
print(jsonCourse)


{'department': 'CS', 'course_number': 101, 'start_date': datetime.date(2025, 1, 23), 'end_date': datetime.date(2025, 9, 23)}
{
  "start_date": "2025-01-23",
  "end_date": "2025-09-23"
}
{
  "department": "CS",
  "course_number": 101
}


In [6]:
from datetime import date
from pydantic import BaseModel, model_validator

class ScheduledCourse(BaseModel):
    department: str = "undefined"
    course_number: int
    start_date: date = date.today()
    end_date: date
    
    @model_validator(mode="after")
    def validate_dates(self):
        if self.start_date > self.end_date:
            raise ValueError("Start date must be before end date.")
        return self

        
course = ScheduledCourse(
    course_number=101,
    end_date="2026-09-23"
)

# serialize only values that were set explicitly and omit defaults
jsonCourse = course.model_dump_json(indent=2, exclude_unset=True)
print(jsonCourse)

{
  "course_number": 101,
  "end_date": "2026-09-23"
}


In [9]:
course = ScheduledCourse(
    course_number=101,
    end_date="2026-01-01",
    department="undefined"
)

# prints department because it was set
jsonCourse = course.model_dump_json(indent=2, exclude_unset=True)
print(jsonCourse)
# does not print department because it has a default value
jsonCourse = course.model_dump_json(indent=2, exclude_defaults=True)
print(jsonCourse)

{
  "department": "undefined",
  "course_number": 101,
  "end_date": "2026-01-01"
}
{
  "course_number": 101,
  "end_date": "2026-01-01"
}


JSON Schema

In [10]:
from pydantic import BaseModel, model_validator
from datetime import date
import json

class ScheduleCourse(BaseModel):
    department: str = "undefined"
    course_number: int
    start_date: date = date.today()
    end_date: date

    @model_validator(mode="after")
    def validate_dates(self):
        if self.start_date > self.end_date:
            raise ValueError("Start date must be before end date.")
        return self

schema = ScheduleCourse.model_json_schema()
print(json.dumps(schema, indent=2))

{
  "properties": {
    "department": {
      "default": "undefined",
      "title": "Department",
      "type": "string"
    },
    "course_number": {
      "title": "Course Number",
      "type": "integer"
    },
    "start_date": {
      "default": "2025-02-07",
      "format": "date",
      "title": "Start Date",
      "type": "string"
    },
    "end_date": {
      "format": "date",
      "title": "End Date",
      "type": "string"
    }
  },
  "required": [
    "course_number",
    "end_date"
  ],
  "title": "ScheduleCourse",
  "type": "object"
}


In [12]:
from typing import List, Optional

class Item(BaseModel):
    name: str
    description: Optional[str] = None
    
class Order(BaseModel):
    id: int
    items: List[Item]

schema = Order.model_json_schema()
print(json.dumps(schema, indent=2))

{
  "$defs": {
    "Item": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        },
        "description": {
          "anyOf": [
            {
              "type": "string"
            },
            {
              "type": "null"
            }
          ],
          "default": null,
          "title": "Description"
        }
      },
      "required": [
        "name"
      ],
      "title": "Item",
      "type": "object"
    }
  },
  "properties": {
    "id": {
      "title": "Id",
      "type": "integer"
    },
    "items": {
      "items": {
        "$ref": "#/$defs/Item"
      },
      "title": "Items",
      "type": "array"
    }
  },
  "required": [
    "id",
    "items"
  ],
  "title": "Order",
  "type": "object"
}


Deserialization

In [18]:
from pydantic import BaseModel, EmailStr, ValidationError
from typing import Optional

class User(BaseModel):
    name: str
    age: int
    email: Optional[EmailStr] = None
    
user_data = '{"name": "Alice", "age": 22, "email": "alice@example.com"}'
try:
    user = User.model_validate_json(user_data)
    print(user)
except ValidationError as ex:
    print(ex)

name='Alice' age=22 email='alice@example.com'
